In [1]:
%load_ext autoreload
%autoreload 2

This notebook will closely follow the [Simulating Molecules using VQE Qiskit Tutorial](https://qiskit.org/textbook/ch-applications/vqe-molecules.html). 
$$\newcommand{\ket}[1]{|{#1}\rangle}\newcommand{\bra}[1]{\langle{#1}|}$$

## Single Qubit VQE

Our goal is to estimate the ground state energy $\lambda_{\text{min}}$ of some Hamiltonian $H$. 

By the variational principle, we know the below is true for some quantum state $|\psi (\vec{\theta})\rangle$ of norm 1, where $\vec{\theta}$ represent parameters that we can set to realize any valid quantum state.

$$ \begin{align}\langle \psi(\vec{\theta})  | H | \psi(\vec{\theta}) \rangle &= \sum_{\lambda_i,\lambda_j} \langle \psi(\vec{\theta}) | \psi_{\lambda_i} \rangle \langle \psi_{\lambda_i} | H | \psi_{\lambda_j} \rangle \langle \psi_{\lambda_j} | \psi(\vec{\theta}) \rangle \\
&= \sum_{\lambda} \lambda |\langle \psi_{\lambda} | \psi(\vec{\theta}) \rangle |^2 \\
&\geq \lambda_{\text{min}} \underbrace{\sum_{\lambda}  |\langle \psi_{\lambda} | \psi(\vec{\theta}) \rangle|^2}_{1} = \lambda_{\text{min}}\end{align}$$ 

We also know that for $|\psi (\vec{\theta_{\text{min}}})\rangle = |\psi_{\text{min}}\rangle$ (the ground state eigenfunction), we satisy the above inequality. 

Thus, we must
1. Find some way to construct an eigenfunction $|\psi (\vec{\theta})\rangle$.
2. Optimize over the $\vec{\theta}$ to minimize $\langle \psi(\vec{\theta})  | H | \psi(\vec{\theta}) \rangle$ and find the lowest upper bound to $\lambda_{\text{min}}$. 

With this motivation in mind, let's begin with the simplest case of a 1 qubit Hilbert space. Here, we can access any arbitrary quantum state by applying the $U_3$ gate on an arbitrary intial state $\ket{\psi_0}$, namely $\ket{\psi (\vec{\theta})} = U_3(\theta_1,\theta_2, \theta_3)\ket{\psi_0}$ where 

$$\begin{align}U_3(\vec{\theta}) &= \begin{pmatrix} \cos\left(\frac{\theta_1}{2}\right) & -e^{i \theta_3} \sin\left(\frac{\theta_1}{2}\right)\\e^{i\theta_2}\sin\left(\frac{\theta_1}{2}\right)& e^{i\theta_3 + i\theta_2}\cos\left(\frac{\theta_1}{2}\right)\end{pmatrix}\end{align}$$

Now that we have defined $\ket{\psi (\vec{\theta})}$ we can minimize the $\langle \psi(\vec{\theta})  | H | \psi(\vec{\theta}) \rangle$ matrix over $\vec{\theta}$. The classical optimizer we will use to do this will first use the _Constrained Optimization by Linear Approximation optimizer_ (COBYLA) optimizer in the noiseless case, and the _Simultaneous Perturbation Stochastic Approximation_ (SPSA) optimizer for a noisy simulator. 

### Example _(ground state energy of random Hermitian)_

#### measuring < psi | H | psi>

In [38]:
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.operators import PauliExpectation, CircuitStateFn, CircuitSampler, StateFn
from qiskit.aqua.operators import MatrixExpectation, AerPauliExpectation
from qiskit.aqua.operators import X, Y, Z, I
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
import numpy as np

In [39]:
# H2-molecule Hamiltonian 
op =  (-1.0523732 * I^I) + (0.39793742 * I^Z) + (-0.3979374 * Z^I) \
    + (-0.0112801 * Z^Z) + (0.18093119 * X^X)

In [40]:
# define the state you w.r.t. which you want the expectation value
psi = QuantumCircuit(2)
# convert to a state
psi = CircuitStateFn(psi)

In [41]:
# "Mathematical" < psi | O | psi > 
print('Math:', psi.adjoint().compose(op).compose(psi).eval().real)

Math: -1.06365328


In [42]:
# Shot-based Results

backend = Aer.get_backend('qasm_simulator') 
q_instance = QuantumInstance(backend, shots=1024)

# define the state to sample
measurable_expression = StateFn(op, is_measurement=True).compose(psi) 

# convert to expectation value
expectations = {}
expectations['shots'] = PauliExpectation().convert(measurable_expression)
expectations['aer'] = AerPauliExpectation().convert(measurable_expression)
expectations['matrix'] = MatrixExpectation().convert(measurable_expression)

samplers = {}
for label, expectation  in expectations.items():
    samplers[label] = CircuitSampler(q_instance).convert(expectations[label]) 

# evaluate
for label, sampler  in samplers.items():
    print(label + ": ", sampler.eval().real)  

shots:  -1.0696607609179685
aer:  -1.06365328
matrix:  -1.06365328


#### optimization using 1 qubit

In [43]:
backend = Aer.get_backend('qasm_simulator') 
q_instance = QuantumInstance(backend, shots=1024)

# # H2-molecule Hamiltonian 
# op =  (-1.0523732 * I^I) + (0.39793742 * I^Z) + (-0.3979374 * Z^I) \
#     + (-0.0112801 * Z^Z) + (0.18093119 * X^X)
np.random.seed(0)
rand_coeff = np.random.randn(4)
op = rand_coeff[0]*I + rand_coeff[1]*X + rand_coeff[2]*Y + rand_coeff[3]*Z
op_np = op.to_matrix()

def get_measurable_expression(params):
    qr = QuantumRegister(1, name="q")
    psi_qc = QuantumCircuit(qr)
    psi_qc.u(params[0], params[1], params[2], qr[0])
    psi_qc = CircuitStateFn(psi_qc)
    return StateFn(op, is_measurement=True).compose(psi_qc) 
    
def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    measurable_expression = get_measurable_expression(params)
    expectation = AerPauliExpectation().convert(measurable_expression)
    sampler = CircuitSampler(q_instance).convert(expectation)
    return sampler.eval().real

print(objective_function([0.1,.1,2]))

4.04325469422588


In [47]:
from qiskit.aqua.components.optimizers import COBYLA

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)

# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)


# Obtain the output distribution using the final parameters
lambda_min = objective_function(ret[0])


print("Parameters Found: ", ret[0])
print("Minimum Energy (vqe): ", lambda_min)

v,w = np.linalg.eigh(op_np)
print("Minumum Energy (exact): ", v[0])

print("Percent difference: ", (lambda_min - v[0])/np.abs(v[0])*100)

Parameters Found:  [ 3.58252811  1.18273116 -0.01819846]
Minimum Energy (vqe):  -0.7137806068776886
Minumum Energy (exact):  -0.7137806111073199
Percent difference:  5.925674144183837e-07
